In [1]:
import requests
from selenium import webdriver
from webdriver_manager.utils import chrome_version
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import chromedriver_binary
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 

# オブジェクトの作製

In [2]:
#県ごとに分ける

In [6]:
class Get_jinzai:
    def __init__(self):
        self.url =  "https://jinzai.hellowork.mhlw.go.jp/JinzaiWeb/GICB101010.do?screenId=GICB101010&action=initDisp"
        
        #webdriverのoptionを設定
        self.op = Options()
        # --headlessだけではOSによって動かない、プロキシが弾かれる、
        # CUI用の省略されたHTMLが帰ってくるなどの障害が出ます。
        # 長いですが、これら6行あって最強かつどんな環境でも動きますので、必ず抜かさないようにしてください。
        self.op.add_argument("no-sandbox")
        self.op.add_argument("--disable-extensions")
        self.op.add_argument("--headless")
        self.op.add_argument('--disable-gpu')  
        self.op.add_argument('--ignore-certificate-errors')
        self.op.add_argument('--allow-running-insecure-content')
        self.op.add_argument('--disable-web-security')
        self.op.add_argument('--disable-desktself.op-notifications')
        self.op.add_argument("--disable-extensions")
        self.op.add_argument('--lang=ja')
        # self.op.add_argument('--blink-settings=imagesEnabled=false')
        self.op.add_argument('--disable-dev-shm-usage')
        self.op.add_argument('--proxy-server="direct://"')
        self.op.add_argument('--proxy-bypass-list=*')    
        self.op.add_argument('--start-maximized')
        
        #dfのカラムをあらかじめ設定
        self.col = ["許可・受理番号\n／許可年月日・\n届出受理年月日", "事業主名称/\n事業所名称","会社url", "事業所所在地/\n電話番号", "得意とする職種", "労使協定の締結", '各種情報', '備考']

        #pageのスタートを選択する為の設定
        self.judge=True
        
        #空のdataframeを作製
        self.df = pd.DataFrame(index=[],  columns=self.col)
    
    #始めのログイン
    def login(self):
        driver = webdriver.Chrome(chrome_options=self.op)
        driver.get(self.url)
        time.sleep(5)
        #派遣業者
        haken = driver.find_element_by_xpath('//*[@id="button_area"]/dt[1]/a')
        haken.click()
        time.sleep(5)
        #各種チェックボックス

        roudo = driver.find_element_by_id("ID_cbJigyoshoKbnHan1")
        tokutei = driver.find_element_by_id("ID_cbJigyoshoKbnChi1")
        zenkoku = driver.find_element_by_id("ID_cbZenkoku1")
        check = [roudo,tokutei,zenkoku]

        for i in check:
            i.click()
            
        # 検索
        search = driver.find_element_by_id("id_btnSearch")
        search.click()
        time.sleep(5)
        return driver
    
    # 1ページ分のループ
    def get_table(self,driver):
        #テーブルの親要素 21の要素　始めの一つはcolmun
        tp= driver.find_elements_by_xpath('//*[@id="search"]/tbody/tr')
        list_fin=[]
        for i,j in enumerate(tp):
            tc = j.find_elements_by_tag_name("td")
            non=[]
            for k,n in enumerate(tc):
                non.append(n.text)
                if k ==1:
                    try:
                        non.append(n.find_element_by_tag_name("a").get_attribute("href"))
                    except:
                        non.append("")
            list_fin.append(non)
            #ここで一つリストが作成される
        list_fin = pd.DataFrame(list_fin,columns = self.col).iloc[1:,:]
        return list_fin,driver
    
            
        #クリック処理 1loop文
    def loop(self,driver,instance,finish):
        concat = self.df
        judge = self.judge
        while judge==True:
            try:
                page = driver.find_elements_by_xpath('//*[@id="search_area"]/table/tbody/tr[5]/td/table[2]/tbody/tr/td/table/tbody/tr/td')
                #もし選択したページより前だったらループを実行
                if int(page[1].text)<=finish:
                    page = driver.find_elements_by_xpath('//*[@id="search_area"]/table/tbody/tr[5]/td/table[2]/tbody/tr/td/table/tbody/tr/td')
                    #コレで10page文のデータプラス1回ジャンプできる
                    for i in range(len(page)-2):
                        #データの処理
                        ll, driver = instance.get_table(driver)
                        concat = pd.concat([concat,ll])
                        #データの処理が完了した後クリック動作に移る ここでsleepを入れないとクリックの挙動がおかしくなる
                        time.sleep(5)
                        page = driver.find_elements_by_xpath('//*[@id="search_area"]/table/tbody/tr[5]/td/table[2]/tbody/tr/td/table/tbody/tr/td')
                        #>>ボタンを押すと11のように下一桁が１になるから
                        sinko = int(page[1].text)+i
                        print("進行度:{}".format(sinko))
                        page[i+2].click()
                        time.sleep(2)
                #選択したページより後になったらループを終了
                elif int(page[1].text) > finish:
                    print("進行度{},処理を停止します".format(page[1].text))
                    judge =  False
            #時間がなくて処理を完了したときに出す。
            except KeyboardInterrupt:
                judge = False
                page = driver.find_elements_by_xpath('//*[@id="search_area"]/table/tbody/tr[5]/td/table[2]/tbody/tr/td/table/tbody/tr/td')
                sinko = int(page[1].text)+i
                print("進行度:{},停止が押されましたので処理を停止します。".format(sinko))
            except:
                page = driver.find_elements_by_xpath('//*[@id="search_area"]/table/tbody/tr[5]/td/table[2]/tbody/tr/td/table/tbody/tr/td')
                sinko = int(page[1].text)+i
                print("進行度:{},予期せぬエラーで処理を停止します。".format(sinko))
        return concat,driver

    #途中のページからスタートしたいとき (start:10回刻み)
    def page_move(self,driver,start):
        judge = self.judge
        while judge==True:
            #これで一回10見聞飛べる
            page = driver.find_elements_by_xpath('//*[@id="search_area"]/table/tbody/tr[5]/td/table[2]/tbody/tr/td/table/tbody/tr/td')
            if int(page[1].text) <= start:
                print("進行度:{}".format(page[1].text))
                page[11].click()
                time.sleep(3)
            elif int(page[1].text) > start:
                print("進行度{},ページネーションを停止します".format(page[1].text))
                judge=False
            
        return driver
    
    def check(self,df):
        #Falseが重複なし
        print(df.duplicated(subset='事業主名称/\n事業所名称').value_counts())

In [7]:
jinzai = Get_jinzai()

In [8]:
driver = jinzai.login()
concat, driver = jinzai.loop(driver,jinzai,1000)

/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:37: DeprecationWarning: use options instead of chrome_options


進行度:1
進行度:2
進行度:3
進行度:4
進行度:5
進行度:6
進行度:7
進行度:8
進行度:9
進行度:10
進行度:11
進行度:12
進行度:13
進行度:14
進行度:15
進行度:16
進行度:17
進行度:18
進行度:19
進行度:20
進行度:21
進行度:22
進行度:23
進行度:24
進行度:25
進行度:26
進行度:27
進行度:28
進行度:29
進行度:30
進行度:31
進行度:32
進行度:33
進行度:34
進行度:35
進行度:36
進行度:37
進行度:38
進行度:39
進行度:40
進行度:41
進行度:42
進行度:43
進行度:44
進行度:45
進行度:46
進行度:47
進行度:48
進行度:49
進行度:50
進行度:51
進行度:52
進行度:53
進行度:54
進行度:55
進行度:56
進行度:57
進行度:58
進行度:59
進行度:60
進行度:61
進行度:62
進行度:63
進行度:64
進行度:65
進行度:66
進行度:67
進行度:68
進行度:69
進行度:70
進行度:71
進行度:72
進行度:73
進行度:74
進行度:75
進行度:76
進行度:77
進行度:78
進行度:79
進行度:80
進行度:81
進行度:82
進行度:83
進行度:84
進行度:85
進行度:86
進行度:87
進行度:88
進行度:89
進行度:90
進行度:91
進行度:92
進行度:93
進行度:94
進行度:95
進行度:96
進行度:97
進行度:98
進行度:99
進行度:100
進行度:101
進行度:102
進行度:103
進行度:104
進行度:105
進行度:106
進行度:107
進行度:108
進行度:109
進行度:110
進行度:111
進行度:112
進行度:113
進行度:114
進行度:115
進行度:116
進行度:117
進行度:118
進行度:119
進行度:120
進行度:121
進行度:122
進行度:123
進行度:124
進行度:125
進行度:126
進行度:127
進行度:128
進行度:129
進行度:130
進行度:131
進行度:132
進行度:133
進行度:134
進行度:135
進行度:136
進行度:137
進行度:138
進行度:

In [9]:
concat.to_csv("kigyo_1000.csv")

In [10]:
jinzai.check(concat)

False    19752
True       248
dtype: int64


In [11]:
concat

,許可・受理番号\n／許可年月日・\n届出受理年月日,事業主名称/\n事業所名称,会社url,事業所所在地/\n電話番号,得意とする職種,労使協定の締結,各種情報,備考
1,派01-010028\n昭和63年09月01日,株式会社ツーリストサービス北海道\n株式会社ツーリストサービス北海道,,北海道札幌市中央区北3条西2-2-1 日通札幌ビル7階\n011-222-6097,,,,
2,派01-010033\n平成01年10月01日,株式会社ケイ・イー・パワースタッフ\n株式会社ケイ・イー・パワースタッフ,,北海道札幌市中央区北一条東一丁目4番地1 サン経成ビル5階\n011-232-4096,,,,
3,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社,http://www.career-bank.co.jp/,北海道札幌市中央区北5条西5-7 sapporo55 5F\n011-251-3373,,,,
4,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社 函館支店,,北海道函館市五稜郭町1-14 五稜郭114ビル 5階\n0138-55-2002,,,,
5,派01-010050\n平成03年06月01日,キャリアバンク株式会社\nキャリアバンク株式会社 旭川支店,http://www.career-bank.co.jp/,北海道旭川市1条通9-50-3 旭川緑橋第一生命ビル3階\n0166-29-2077,,,,
...,...,...,...,...,...,...,...,...
16,派13-312180\n平成31年01月01日,株式会社シンプリズム\n株式会社シンプリズム,,東京都渋谷区代々木１‐１３‐８ 山野代々木ビル５Ｆ\n03-6694-6652,,,,
17,派13-312181\n平成31年01月01日,有限会社エムズスタッフ\n有限会社エムズスタッフ,,東京都新宿区北新宿一丁目１３番２号 １階\n03-3371-3856,,,,
18,派13-312182\n平成31年01月01日,有限会社ティーエーシー\n有限会社ティーエーシー,,東京都府中市八幡町二丁目２１番地の４ １Ｆ\n042-363-3226,,,,
19,派13-312183\n平成31年01月01日,株式会社アフロ\n株式会社アフロ,,東京都台東区台東一丁目５番２１号 田中ビル２階\n03-3831-6505,,,,
